In [1]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from pypdf import PdfReader
from langchain.document_loaders import PyPDFLoader
from langchain.llms import OpenAI
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI

# SAH:
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

In [ ]:
import os
import json
import pandas as pd
import glob
from openai import OpenAI

In [ ]:
# Get all PDFs inside the tsmc folder
pdf_files = glob.glob("./tsmc/*.pdf")

In [ ]:
# Load & split documents from all PDFs
all_docs = []
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

for pdf in pdf_files:
    loader = PyPDFLoader(pdf)
    data = loader.load()
    texts = text_splitter.split_documents(data)
    all_docs.extend(texts)

In [7]:
# Use Ollama embeddings with a supported model
""" embeddings = OpenAIEmbeddings(model="text-embedding-3-large") """
# SAH:
embeddings = OllamaEmbeddings(model="nomic-embed-text") 

In [ ]:
""" os.environ["OPENAI_API_KEY"] = "sk-XXX" #Replace with your OpenAI API key """

In [ ]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore.from_documents(all_docs, embeddings)

In [ ]:
print(f"✅ Total PDFs: {len(pdf_files)}")
print(f"✅ Total chunks stored: {len(all_docs)}")

✅ Total PDFs: 0
✅ Total chunks stored: 0


In [ ]:
first_chunk = all_docs[102]
print("=== Page Content ===")
print(first_chunk.page_content)
print("=== Metadata ===")
print(first_chunk.metadata)

In [ ]:
#Use GPT-4o

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    openai_api_key=os.environ.get("OPENAI_API_KEY")
)


In [ ]:
#Build RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_store.as_retriever(),
    return_source_documents=True
)

In [ ]:
#Run a query
query = "Tell the key points about TSMC's 3nm technology from the documents."
result = qa_chain({"query": query})

print(result["result"])